In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import urlopen
import time
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import xlwt
import json
from datetime import datetime, timedelta

In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://ebook.kyobobook.co.kr/dig/etc/landing/best")

C:\Users\woosung\AppData\Local\Temp\ipykernel_28336\2341044327.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
index = "kyobo_best_100"

In [8]:
present_date = str(datetime.utcnow() + timedelta(hours=9))[:10]

In [9]:
book_list = []

genre_dict = { 
        18 : "경제경영", # 
        19 : "자기계발", # 경제/비즈니스
        20 : "시/에세이", # 
        21 : "인문", # 
        22 : "종교", #  x
        23 : "소설", # 
        24 : "국어/외국어", # 
        25 : "정치/사회", #  x
        26 : "역사/문화", #  x
        27 : "과학/공학", # 
        28 : "IT/프로그래밍", # 
        29 : "건강/의학", #  
        30 : "가정/생활/요리", # 
        31 : "여행/취미", # 
        32 : "예술/대중문화", # 
        33 : "유아",  # x
        34 : "아동", # x
        35 : "청소년", # x
        36 : "교재/수험서", #
        37 : "외국도서", #
        38 : "매거진" # x
}

sm_100_genre_dict = {
    1 : "문학", #23
    2 : "에세이/산문", #20
    3 : "인문", #21
    4 : "경제/비즈니스", #18,19
    5 : "자연/과학", #27
    6 : "컴퓨터/인터넷", #28
    7 : "외국어", #24
    8 : "가정/생활", #29,30, 31, 32
    9 : "강의지원도서", #36
    10 : "국외ebook" #38
}

#100개씩 보기 선택
driver.find_element(By.ID,"ebookLanding_pageCnt-button").click()
time.sleep(0.5)
    
driver.find_element(By.ID,"ui-id-58").click()
time.sleep(0.5)

bsObject = BeautifulSoup(driver.page_source, 'html.parser')
dates = bsObject.select_one('#ui-id-42').text # 오늘날짜

# 장르 선택
for genre_num in range(18,39):
    driver.find_element(By.ID,"ebookLanding_msc-button").click()
    time.sleep(0.5)
    
    driver.find_element(By.ID,f"ui-id-{genre_num}").click()
    time.sleep(0.5)
    
    bsObject = BeautifulSoup(driver.page_source, 'html.parser')
    
    books = bsObject.select('div.prodDt')
    rank = 0
    for book in books:
        
        rank = rank + 1# 순위
        
        title = book.select_one('strong > h3 > a').text  #책 제목
        
        author = book.select_one('p.prodDt_info > span').text  #저자
        
        publisher = book.select('p.prodDt_info > span')[1].text #출판사
        
        date = book.select('p.prodDt_info > span')[2].text  #등록일
        
        price =book.select_one('div.prodDt_price span b').text # 가격
        
        if(book.select_one('div.prodDt_review > p > b') == None) : #만약 별점이 없으면 0으로
            review = 0
            
        else :
            review = book.select_one('div.prodDt_review > p > b').text #별점
            
        genre=genre_dict[genre_num]
        book_list.append([title,rank,author,publisher,date,price,review,genre,genre,present_date])



In [11]:
df = pd.DataFrame(book_list,columns=["제목","kyobo_순위","kyobo_저자","kyobo_출판사","kyobo_발매일","kyobo_가격","kyobo_평점","kyobo_장르","상명대학교_학술정보관_장르","날짜"])

In [12]:
# 장르 상명대에게 맞추기
df2 = df.replace({'상명대학교_학술정보관_장르' : {'경제경영':'경제/비즈니스','자기계발':'경제/비즈니스','시/에세이':'에세이/산문','소설':'문학','국어/외국어':'외국어','과학/공학':'자연/과학','IT/프로그래밍':'컴퓨터/인터넷','가정/생활/요리':'가정/생활','여행/취미':'가정/생활','예술/대중문화':'가정/생활','교재/수험서':'수험서/자격증','외국도서':'국외ebook'}})

In [13]:
#데이터 전처리
df2['kyobo_발매일'] = df2['kyobo_발매일'].str.replace('.', '')
df2['kyobo_발매일'] = df2['kyobo_발매일'].str.slice(0,8)
df2['kyobo_가격'] = df2['kyobo_가격'].str.replace(',', '')
df2['kyobo_발매일'] = pd.to_datetime(df2['kyobo_발매일'])

C:\Users\woosung\AppData\Local\Temp\ipykernel_28336\3946334573.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['kyobo_발매일'] = df2['kyobo_발매일'].str.replace('.', '')


In [14]:
df3 = df2

In [24]:
df3

,제목,kyobo_순위,kyobo_저자,kyobo_출판사,kyobo_발매일,kyobo_가격,kyobo_평점,kyobo_장르,상명대학교_학술정보관_장르
0,챗GPT,1,반병현,생능북스,2023-02-17,9900,9.4,경제경영,경제/비즈니스
1,레버리지,2,롭 무어,다산북스,2023-02-15,11340,8.8,경제경영,경제/비즈니스
2,주식투자 무작정 따라하기(2023),3,윤재수,길벗,2023-02-03,16560,9.7,경제경영,경제/비즈니스
3,천 원을 경영하라,4,박정부,쌤앤파커스,2022-12-07,10080,9.8,경제경영,경제/비즈니스
4,실전 투자강의,5,앙드레 코스톨라니,미래의창,2022-03-29,8640,9.7,경제경영,경제/비즈니스
...,...,...,...,...,...,...,...,...,...
1457,더그아웃(DUGOUT) 2021년 12월호,43,대단한미디어,대단한미디어,2021-12-03,3600,10,매거진,매거진
1458,자전거생활 2016년 10월호,44,비엘프레스,비엘프레스,2016-10-10,2700,10,매거진,매거진
1459,중급 일본어(EBS 방송교재 2023년 2월),45,원미령,EBS한국교육방송공사,2023-01-26,5040,10,매거진,매거진
1460,POWER ENGLISH(EBS 방송교재 2023년 2월),46,크리스틴 조,EBS한국교육방송공사,2023-01-26,5760,8.2,매거진,매거진


In [16]:
from elasticsearch import Elasticsearch, helpers
import configparser

In [17]:
config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [18]:
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

In [36]:
if es.indices.exists(index = index + present_date):
    pass
else:
    es.indices.create(index = index + present_date, body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "nori": {
                        "tokenizer": "nori_tokenizer"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "제목": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "kyobo_장르": {
                    "type": "keyword"
                },
                "kyobo_저자": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "kyobo_출판사": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "kyobo_발매일": {
                    "type": "date"
                },
                "상명대학교_학술정보관_장르": {
                    "type": "keyword"
                },
                "kyobo_가격": {
                    "type": "integer"
                },
                "kyobo_평점": {
                    "type": "float"
                },
                "날짜": {
                    "type": "date"
                }
            }
        }
    })
                      

In [37]:
for i in range(len(df3)) :
    es.index(index= index + present_date,
         document = {
         "제목" : str(df3.loc[i].제목),
         "kyobo_순위" : str(df3.loc[i].kyobo_순위),
         "kyobo_저자": str(df3.loc[i].kyobo_저자),
         "kyobo_출판사": str(df3.loc[i].kyobo_출판사),
         "kyobo_발매일": str(df3.loc[i].kyobo_발매일),
         "kyobo_가격": str(df3.loc[i].kyobo_가격),
         "kyobo_평점": str(df3.loc[i].kyobo_평점),
         "kyobo_장르": str(df3.loc[i].kyobo_장르),
         "상명대학교_학술정보관_장르": str(df3.loc[i].상명대학교_학술정보관_장르),
         "날짜" : present_date  
 })